In [4]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [5]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


# Helper Functions

In [29]:
def init_weight(shape):
    # shape = [H, W, Depth, Filters]
    # shape is a array 
    weight = tf.random_normal(shape=shape, stddev=0.1)
    return tf.Variable(weight, dtype=tf.float32)

def init_bias(size):
    # size = Number of Filter last layer
    bias = tf.constant(shape=size, value=0.1)
    return tf.Variable(bias, dtype=tf.float32)

def conv2d(x, W):
    # x.shape = [batch, H, W, Depth]
    return tf.nn.conv2d(input=x, filter=W, strides=[1, 1, 1, 1], padding='SAME')

def pooling(x):
    # reduce 75% Every pooling
    return tf.nn.max_pool(x, strides=[1, 2, 2, 1], ksize=[1, 2, 2, 1], padding='SAME')

# Layers

In [41]:
def conv_layer(input_image, shape):
    W = init_weight(shape)
    b = init_weight([shape[3]])

    return tf.nn.relu(conv2d(input_image, W) + b)

def fullly_connected_layer(x, size):
    x_shape = int(x.get_shape()[1])
    W = init_weight([x_shape, size])
    b = init_weight([size])
    return tf.matmul(x, W) + b 

# Place Holder

In [45]:
x = tf.placeholder(dtype=tf.float32, shape=[None, 784])
y_true = tf.placeholder(dtype=tf.float32, shape=[None, 10])
hold_prob = tf.placeholder(dtype=tf.float32)

image = tf.reshape(tensor=x, shape=[-1, 28, 28, 1])

convo_1 = conv_layer(image, [6, 6, 1, 32])
pooling_1 = pooling(convo_1)

convo_2 = conv_layer(pooling_1, [6, 6, 32, 64])
pooling_2 = pooling(convo_2)

flat = tf.reshape(pooling_2, [-1, 7 * 7 * 64]) # Could be error
# Add drop out
dropout = tf.nn.dropout(x=flat, keep_prob=hold_prob)
y_pred = fullly_connected_layer(x=dropout, size=10)

# Train

In [51]:
loss = tf.nn.softmax_cross_entropy_with_logits(labels=y_true, logits=y_pred)
optimizer = tf.train.AdamOptimizer(learning_rate=0.0001)
train = optimizer.minimize(loss)

init = tf.global_variables_initializer()
steps = 100000
with tf.Session() as sess:
    sess.run(init)
    
    for i in range(steps):
        x_batch, y_batch = mnist.train.next_batch(50)
        sess.run(train, feed_dict={x:x_batch, y_true:y_batch, hold_prob:0.5})
        
        if i % 100 != 0:
            continue
        
        print ('Currently Step {}'.format(i))
        matches = tf.equal(x=tf.argmax(y_pred, axis=1), y=tf.argmax(y_true, axis=1))
        acc = tf.reduce_mean(tf.cast(matches, tf.float32))
        result = sess.run(acc, feed_dict={x:mnist.test.images, y_true:mnist.test.labels, hold_prob:1.0})
        print ('Accuracy is:', result)

Currently Step 0
Accuracy is: 0.0796
Currently Step 100
Accuracy is: 0.5241
Currently Step 200
Accuracy is: 0.72
Currently Step 300
Accuracy is: 0.8211


KeyboardInterrupt: 